# NCEI
<br>

- 월평균 활용
- https://apihub.kma.go.kr/
    - 세계기상 > NCEI 관측.통계 > 3. 전세계 지상관측(월통계) : 1763년~2022년
    - ※ NCEI(National Centers for Environmental Information, 국립환경정보센터)는 미국 NOAA의 소속된 기관으로 전세계 기상기후데이터를 수집·제공합니다.
    - ※ 모든 자료가 2022년 12월 까지 존재합니다.
## 참고 자료
- document 파일
    - 자료 소개서
        - GSOM_readme.txt
    - 자료 설명서
        - GSOM_documentation.pdf
    - 연/월 통합 상세 설명서
        - GSOM_GSOY_Description_Document_v1.0.2_20200219.pdf
- 지점정보
    - 링크 내용은 동일함
    - https://www.ncei.noaa.gov/pub/data/noaa/isd-history.txt(내용확인)
    - https://www.ncei.noaa.gov/pub/data/noaa/isd-history.csv(파일다운로드)
- 국가 코드
    - Country list
        - 국가명과 국가 2글자 표현 짝 정리된 것
        - 후에 요청인자 중 stn 구성요소인 지점코드값의 국가 부분을 해석하는데에 사용
        - 293개국 존재
- 파일 리스트
    - File list
        - 파일목록
        - 파일명과 파일 크기가 나열됨
        - 후술될 요청인자 중 stn의 구성요소인 지점코드 값으로 활용할 수 있음
            - 국가 2글자, 망구분 1글자, 지점번호 9글자를 가져올 소스
        - 122236개 있음
- 실제 파일 리스트
    - data_climate/gsom-latest 내부에 존재하는 파일들
    - 122041개 있음

## 코드 목적
- 사용 데이터: NCEI의 GSOM(전세계 월별 요약) 기후 데이터
- 데이터 위치: jihye 브랜치에 data_climate 폴더 내 gsom-latest csv 파일들
- 필요한 일 (우선순위 순서)
    1. Wine Data / OIV 폴더OIV_wine_producing_country_and_region.xlsx에 올라와있는 20개국(country_20)을 기준으로 데이터 정합성이 가장 좋은 해(year_1)를 고르기
        - 결과물
            1. year_1을 기준으로 country_20 데이터 누적한 파일
            2. year_1을 기준으로 나머지 국가 데이터 누적한 파일
    2. Wine Data / OIV 폴더OIV_wine_producing_country_and_region.xlsx에 올라와있는 20개국(country_20)을 기준으로 데이터 정합성이 가장 좋은 해 연속 5년(year_5)을 고르기
        - 결과물
            3. year_5을 기준으로 country_20 데이터 누적한 파일
            4. year_5을 기준으로 나머지 국가 데이터 누적한 파일
    3. 기후 데이터 EDA 아이디어 정리
    4. Wine Data / OIV 폴더OIV_wine_producing_country_and_region.xlsx에 올라와있는 20개국(country_20)을 기준으로 데이터 정합성이 가장 좋은 해 연속 10년(year_10)을 고르기
        - 결과물
            5. year_10을 기준으로 country_20 데이터 누적한 파일
            6. year_10을 기준으로 나머지 국가 데이터 누적한 파일
- 정합성 체크리스트
    - 12개월이 온전히 들어있는지
    - 결측치가 없는지
    - 가장 최신인지
    - 포도 재배 결과가 좋은 21, 20년도일 경우가 선호됨
- 20개국을 정합성 체크하는 기준으로 제한하는 이유
    - 모델 학습에 활용되므로 중요순위가 높다는 자체적 가중치를 둠
    - 역으로 기후 데이터에 정합성이 떨어지는 국가라면, 까다로운 품종 포도를 재배하는데에 적합한 정도의 기술력, 관심도가 떨어진다고 볼 수도 있음
    - 두 명이 주말 안에 못 함
    - 다만 20개국 정합성이 비슷한 두 해가 있고, 나머지 국가들의 데이터 정합성이 크게 차이난다면 차후 논의해볼 문제
- 활용 코드
    - climate_EDA.ipynb
        - open_csv('국가명') 함수를 이용하면 원하는 국가가 있는 모든 지역을 확인할 수 있음
        - open_csv_year('국가리스트', 연도) 함수를 이용하면 위 open_csv('국가명')을 활용해 불러온 국가가 있는 모든 지역 csv에서 특정 연도를 추출, 누적한 df를 얻을 수 있음

## 활용 library

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
dfFiletoCountry = pd.read_csv('../ML_wine/FiletoCountry.csv')
dfFiletoCountry

,RealFileNM,ID,COUNTRY NAME,Name
0,ACW00011604.csv,AC,ANTIGUA AND BARBUDA,ACW00011604
1,ACW00011647.csv,AC,ANTIGUA AND BARBUDA,ACW00011647
2,AE000041196.csv,AE,NaN,AE000041196
3,AEM00041194.csv,AE,NaN,AEM00041194
4,AEM00041217.csv,AE,NaN,AEM00041217
...,...,...,...,...
122036,ZI000067969.csv,ZI,ZIMBABWE,ZI000067969
122037,ZI000067975.csv,ZI,ZIMBABWE,ZI000067975
122038,ZI000067977.csv,ZI,ZIMBABWE,ZI000067977
122039,ZI000067983.csv,ZI,ZIMBABWE,ZI000067983


## 파일 목록 중에 활용할 csv를 불러오는 함수만들기
- 필요한 것
    - 적용 국가
        - Wine Data / OIV 폴더OIV_wine_producing_country_and_region.xlsx 파일 참고
    - 비적용 국가
        - 위 국가 뺀 나머지 국가들 정리

### 국가명 리스트 정리
- 적용국가
    - country_32
- 비적용 국가
    - countryNameList에서 country_32를 뺀 것

In [3]:
# 데이터 정합성 기준 국가
wineCountry = pd.read_excel('OIV_wine_producing_country_and_region.xlsx', engine = 'openpyxl')
# Null값은 제외해야 함
country_32 = list(wineCountry['Country'].dropna().unique()[:-1])
print(country_32.index('\xa0Japan'))
country_32[18] = 'Japan'

print(country_32)
print(len(country_32))

country_32 = [country.strip().upper() for country in country_32]

print(country_32)

c:\Users\roha2\AppData\Local\Programs\Python\Python39\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


18
['Argentina', 'Australia', 'Austria', 'Belgium', 'Bulgaria', 'Brazil', 'Czech Republic', 'Switzerland', 'Chile', 'Croatia', 'Cyprus', 'Germany', 'Spain', 'France', 'Georgia', 'Greece', 'Hungary', 'Italy', 'Japan', 'Luxemburg', 'Morocco', 'Moldova', 'New Zealand', 'Peru', 'Portugal', 'Romania', 'Serbia', 'Slovakia', 'Slovenia', 'Republic of Türkiye', 'Uruguay', 'South Africa']
32
['ARGENTINA', 'AUSTRALIA', 'AUSTRIA', 'BELGIUM', 'BULGARIA', 'BRAZIL', 'CZECH REPUBLIC', 'SWITZERLAND', 'CHILE', 'CROATIA', 'CYPRUS', 'GERMANY', 'SPAIN', 'FRANCE', 'GEORGIA', 'GREECE', 'HUNGARY', 'ITALY', 'JAPAN', 'LUXEMBURG', 'MOROCCO', 'MOLDOVA', 'NEW ZEALAND', 'PERU', 'PORTUGAL', 'ROMANIA', 'SERBIA', 'SLOVAKIA', 'SLOVENIA', 'REPUBLIC OF TÜRKIYE', 'URUGUAY', 'SOUTH AFRICA']


In [4]:
# NCEI 데이터 내 표기 국가명 변수
# Null값은 제외해야 함
countryNameList = list(dfFiletoCountry['COUNTRY NAME'].dropna().unique())
countryNameList = [country.strip() for country in countryNameList]
print(countryNameList)

['ANTIGUA AND BARBUDA', 'AFGHANISTAN', 'ALGERIA', 'AZERBAIJAN', 'ALBANIA', 'ARMENIA', 'ANGOLA', 'AMERICAN SAMOA', 'ARGENTINA', 'AUSTRALIA', 'AUSTRIA', 'ANTARCTICA', 'BAHRAIN', 'BARBADOS', 'BOTSWANA', 'BERMUDA', 'BELGIUM', 'BAHAMAS THE', 'BANGLADESH', 'BELIZE', 'BOSNIA AND HERZEGOVINA', 'BOLIVIA', 'BURMA', 'BENIN', 'BELARUS', 'SOLOMON ISLANDS', 'BRAZIL', 'BULGARIA', 'BRUNEI', 'BURUNDI', 'CANADA', 'CAMBODIA', 'CHAD', 'SRI LANKA', 'CONGO', 'ZAIRE', 'CHINA', 'CHILE', 'CAYMAN ISLANDS', 'COCOS (KEELING) ISLANDS', 'CAMEROON', 'COLOMBIA', 'NORTHERN MARIANA ISLANDS', 'COSTA RICA', 'CENTRAL AFRICAN REPUBLIC', 'CUBA', 'CAPE VERDE', 'COOK ISLANDS', 'CYPRUS', 'DENMARK', 'DOMINICA', 'DOMINICAN REPUBLIC', 'ECUADOR', 'EGYPT', 'IRELAND', 'EQUATORIAL GUINEA', 'ESTONIA', 'ERITREA', 'EL SALVADOR', 'ETHIOPIA', 'EUROPA ISLAND', 'CZECH REPUBLIC', 'FRENCH GUIANA', 'FINLAND', 'FIJI', 'FALKLAND ISLANDS (ISLAS MALVINAS)', 'MICRONESIA, FEDERATED STATES OF', 'FRENCH POLYNESIA', 'FRANCE', 'FRENCH SOUTHERN AND ANTAR

In [5]:
# NCEI 국가 리스트에서 기준 국가 빠진 리스트
a = len(countryNameList)
b = len(country_32)
# NCEI 국가 리스트에서 32개 뺀 값
print(a-b)

set_a = set(countryNameList)
set_b = set(country_32)

# b 차집합 a 결과
print(len(set_b.difference(set_a)))

# a 차집합 b 결과
print(set_a.difference(set_b))

# a 차집합 b 결과값에서 국가명 표기가 다른 곳 2군데를 찾음
# 룩셈부르크, 터키
# 추후에 작업할 때 참고해야 함
print('\n국가명 표기가 다른 곳')
print('LUXEMBURG' + ' > NCEI: ' + 'LUXEMBOURG')
print('REPUBLIC OF TÜRKIYE' + ' > NCEI: ' +  'TURKEY')
country_others = list(set_a.difference(set_b))
country_others.remove('LUXEMBOURG')
country_others.remove('TURKEY')

print(country_32.index('REPUBLIC OF TÜRKIYE'))
print(country_32.index('LUXEMBURG'))

country_32[29] = 'TURKEY'
country_32[19] = 'LUXEMBOURG'

len(country_others)

183
2
{'BENIN', 'BOLIVIA', 'PAPUA NEW GUINEA', 'JAMAICA', 'NEPAL', 'SVALBARD', 'UNITED STATES', 'UZBEKISTAN', 'MAURITANIA', 'BAHAMAS THE', 'LITHUANIA', 'FINLAND', 'FRENCH GUIANA', 'TROMELIN ISLAND', 'CHINA', 'BOTSWANA', 'SUDAN', 'BELARUS', 'ZAIRE', 'IRAN', 'ZIMBABWE', 'MOZAMBIQUE', 'TAJIKISTAN', 'LIBERIA', 'WESTERN SAHARA', 'SINGAPORE', 'LATVIA', 'ANTARCTICA', 'MADAGASCAR', 'DOMINICAN REPUBLIC', 'PITCAIRN ISLANDS', 'NETHERLANDS', 'PAKISTAN', 'RWANDA', 'GUINEA', 'LIBYA', 'MACEDONIA', 'VENEZUELA', 'TONGA', 'PHILIPPINES', 'BRITISH INDIAN OCEAN TERRITORY', 'PALAU - TRUST TERRITORY OF THE PACIFIC ISLANDS', 'MALDIVES', 'GUINEA-BISSAU', 'CENTRAL AFRICAN REPUBLIC', 'CAYMAN ISLANDS', 'POLAND', 'FRENCH POLYNESIA', 'FALKLAND ISLANDS (ISLAS MALVINAS)', 'SRI LANKA', 'SIERRA LEONE', 'VIRGIN ISLANDS (U.S.)', 'GIBRALTAR', 'GUAM', 'IRELAND', 'SYRIA', 'THAILAND', 'MAURITIUS', 'ANGOLA', 'BAHRAIN', 'COLOMBIA', 'SWEDEN', 'CHRISTMAS ISLAND', 'PUERTO RICO', 'EUROPA ISLAND', 'TOGO', 'GABON', 'TRINIDAD AND TOB

183

## 국가별 함수
- 국가별로 csv 파일을 불러와서 df_지점코드 라는 변수명에 저장한다
- df_지점코드 형식의 변수명은 varList라는 변수명 리스트에 저장된다
- varList가 반환된다.
- varList에 들어있는 i번째 변수명을 변수로 활용하는 방법
    - globals()[varList[i])

In [6]:
url = 'C:/Users/roha2/AppData/Local/Temp/_AZTMP0_/gsom-latest/'
def open_csv(country = 'korea, south'):
    
    # 변수명 리스트
    varList = []
    
    # COUNTRY NAME에 country 인수가 포함되어있는 행을 추출
    csvList = dfFiletoCountry[dfFiletoCountry['COUNTRY NAME'].str.contains((country.upper())) == True]
    # 국가별 기상 관측 장소 갯수
    cnt = len(csvList)
    # 추출된 행의 RealFileNM값들을 리스트로 저장
    stnList = csvList['RealFileNM'].values
    
    # varList = df_KS#######.csv 모음
    for STN in stnList:
        varNM = 'df_{}'.format(STN)
        

        globals()[varNM] = pd.read_csv(url + STN)
        varList.append(varNM)
    
    print('변수 갯수: ', cnt)
       
    return varList

In [7]:
# 변수명은 상관X
koreaList = open_csv()

변수 갯수:  56


## 특정 연도 데이터 추출 및 누적 함수
- 변수명이 들어있는 리스트를 활용해 특정연도 데이터를 추출하여 누적
- dfTMP에 활용 df를 순차적으로 할당한다.
- or_args(\*args)함수를 활용해 복수의 연도를 적용한다.
    - 이때 연도는 문자타입으로 나열함
    - ex) or_args(varList, '2022', '2021', ...)
- df변수에 dfTMP에서 특정연도 추출한 행을 누적한다.
- df를 반환한다.

In [14]:
def open_csv_year(varList, *year):
    
    flag = False
    
    # 국가별 df 변수를 순서대로 불러 df_TMP에 저장
    for i in range(len(varList)):
        
#         print('적용 df: ', varList[i])
        dfTMP = globals()[varList[i]]
        
        # 지정된 연도를 |로 묶어서 str 변수를 반환
        def or_args(*args):
            items = ''

            for num, item in enumerate(args):
                if num == len(args) - 1:
                    items += item
                else:
                    items += item + '|'

            return items

        # 활용 연도
        Years = or_args(*year)
        tmp = dfTMP[dfTMP['DATE'].str.contains(Years)]
        # csv 처음 불러왔을 때
        if flag == False:
#             print('0')
            # 결과 df 변수에 Years가 포함된 데이터만 추출
            df = tmp
            # flag를 변경하여 이 if문에 다시 들어오지 않는다.
            flag = True
            print('길이: ', len(df))
        else:
#             print('1')
            # 결과 df와 지금 df의 Years가 포함된 데이터를 이어붙인다.
            df = pd.concat([df, tmp], axis = 0)
            print('추가 행 길이: ', len(tmp))
            
    df.reset_index(inplace = True, drop = True)
    print(Years, str(varList))
    
    return df

In [9]:
dfKorea = open_csv_year(koreaList, '2022', '2021')
len(dfKorea)

길이:  3
추가 행 길이:  3
추가 행 길이:  3
추가 행 길이:  3
추가 행 길이:  3
추가 행 길이:  3
추가 행 길이:  4
추가 행 길이:  3
추가 행 길이:  3
추가 행 길이:  3
추가 행 길이:  3
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
2022|2021 ['df_KS000047112.csv', 'df_KS000047115.csv', 'df_KS000047165.csv', 'df_KSM00047101.csv', 'df_KSM00047105.csv', 'df_KSM00047108.csv', 'df_KSM00047133.csv', 'df_KSM00047138.csv', 'df_KSM00047159.csv', 'df_KSM00047168.csv', 'df_KSM00047184.csv', 'df_KSW00043201.csv', 'df_KSW00043206.csv', 'df_KSW00043207.csv', 'df_KSW0004320

34

In [10]:
dfKorea

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,CDSD,CDSD_ATTRIBUTES,CLDD,CLDD_ATTRIBUTES,...,DYTS,DYTS_ATTRIBUTES,EMSN,EMSN_ATTRIBUTES,SNOW,SNOW_ATTRIBUTES,WDFG,WDFG_ATTRIBUTES,WSFG,WSFG_ATTRIBUTES
0,KS000047112,2022-05,37.467,126.633,70.0,"INCHEON, KS",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,KS000047112,2022-06,37.467,126.633,70.0,"INCHEON, KS",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KS000047112,2022-08,37.467,126.633,70.0,"INCHEON, KS",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,KS000047115,2022-05,37.483,130.900,220.0,"ULLEUNGDO, KS",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,KS000047115,2022-06,37.483,130.900,220.0,"ULLEUNGDO, KS",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,KS000047115,2022-08,37.483,130.900,220.0,"ULLEUNGDO, KS",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,KS000047165,2022-05,34.817,126.383,39.0,"MOKPO, KS",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,KS000047165,2022-06,34.817,126.383,39.0,"MOKPO, KS",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,KS000047165,2022-08,34.817,126.383,39.0,"MOKPO, KS",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,KSM00047101,2022-05,37.900,127.733,78.0,"CHUNCHEON, KS",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


---

# !!! 현정님 이곳입니다!!!
- 기준 국가_32를 적용한 데이터의 특정 연도 데이터 확인하기
- 나머지 국가에 특정 연도 데이터 확인하기

In [11]:
# 기준 국가_32 변수
print(country_32)

# NCEI 데이터 내 표기 국가명 변수
print(countryNameList)

# 기준 국가 빠진 나머지 국가
# 정확한 코드 아님, 직관적 이해 위해 적은 것
# country_others = countryNameList - country_20
print(country_others)

['ARGENTINA', 'AUSTRALIA', 'AUSTRIA', 'BELGIUM', 'BULGARIA', 'BRAZIL', 'CZECH REPUBLIC', 'SWITZERLAND', 'CHILE', 'CROATIA', 'CYPRUS', 'GERMANY', 'SPAIN', 'FRANCE', 'GEORGIA', 'GREECE', 'HUNGARY', 'ITALY', 'JAPAN', 'LUXEMBOURG', 'MOROCCO', 'MOLDOVA', 'NEW ZEALAND', 'PERU', 'PORTUGAL', 'ROMANIA', 'SERBIA', 'SLOVAKIA', 'SLOVENIA', 'TURKEY', 'URUGUAY', 'SOUTH AFRICA']
['ANTIGUA AND BARBUDA', 'AFGHANISTAN', 'ALGERIA', 'AZERBAIJAN', 'ALBANIA', 'ARMENIA', 'ANGOLA', 'AMERICAN SAMOA', 'ARGENTINA', 'AUSTRALIA', 'AUSTRIA', 'ANTARCTICA', 'BAHRAIN', 'BARBADOS', 'BOTSWANA', 'BERMUDA', 'BELGIUM', 'BAHAMAS THE', 'BANGLADESH', 'BELIZE', 'BOSNIA AND HERZEGOVINA', 'BOLIVIA', 'BURMA', 'BENIN', 'BELARUS', 'SOLOMON ISLANDS', 'BRAZIL', 'BULGARIA', 'BRUNEI', 'BURUNDI', 'CANADA', 'CAMBODIA', 'CHAD', 'SRI LANKA', 'CONGO', 'ZAIRE', 'CHINA', 'CHILE', 'CAYMAN ISLANDS', 'COCOS (KEELING) ISLANDS', 'CAMEROON', 'COLOMBIA', 'NORTHERN MARIANA ISLANDS', 'COSTA RICA', 'CENTRAL AFRICAN REPUBLIC', 'CUBA', 'CAPE VERDE', 'COO

In [12]:
# country_32 소문자로 변환 
country_32sm = []
for name in country_32:
    country_32sm.append( name.lower())

country_32sm

['argentina',
 'australia',
 'austria',
 'belgium',
 'bulgaria',
 'brazil',
 'czech republic',
 'switzerland',
 'chile',
 'croatia',
 'cyprus',
 'germany',
 'spain',
 'france',
 'georgia',
 'greece',
 'hungary',
 'italy',
 'japan',
 'luxembourg',
 'morocco',
 'moldova',
 'new zealand',
 'peru',
 'portugal',
 'romania',
 'serbia',
 'slovakia',
 'slovenia',
 'turkey',
 'uruguay',
 'south africa']

In [22]:
# open_csv 함수
url = 'C:/Users/roha2/Downloads/gsom-latest/'
def open_csv(country):
    
    # 변수명 리스트
    varList = []

    csvList = dfFiletoCountry[dfFiletoCountry['COUNTRY NAME'].str.contains((country.upper())) == True]
    cnt = len(csvList)
    stnList = csvList['RealFileNM'].values
    
    for STN in stnList:
        varNM = 'df_{}'.format(STN)

        globals()[varNM] = pd.read_csv(url + STN)
        varList.append(varNM)
    
    print('변수 갯수: ', cnt)
       
    return varList


In [42]:
# country_32에 있는 국가명을 다음 함수에 넣어주세요.

# 국가별 데이터 프레임 리스트를 저장할 딕셔너리
data_dict = {}

# country_32에 있는 국가명 for문으로 반복시켜 변수에 할당
# CN = countrt name

for CN in country_32sm[5:]:
    data_dict[CN] = open_csv(CN)

#결과확인

for CN, data in data_dict.items():
    print(f'{CN} 데이터 갯수: {len(data)}')


변수 갯수:  5969
변수 갯수:  12
변수 갯수:  10
변수 갯수:  21
변수 갯수:  14
변수 갯수:  8
변수 갯수:  1120
변수 갯수:  207
변수 갯수:  111
변수 갯수:  105
변수 갯수:  32
변수 갯수:  10
변수 갯수:  104
변수 갯수:  202
변수 갯수:  11
변수 갯수:  20
변수 갯수:  18
변수 갯수:  15
변수 갯수:  25
변수 갯수:  26
변수 갯수:  30
변수 갯수:  3
변수 갯수:  6
변수 갯수:  20
변수 갯수:  58
변수 갯수:  150
변수 갯수:  1166
brazil 데이터 갯수: 5969
czech republic 데이터 갯수: 12
switzerland 데이터 갯수: 10
chile 데이터 갯수: 21
croatia 데이터 갯수: 14
cyprus 데이터 갯수: 8
germany 데이터 갯수: 1120
spain 데이터 갯수: 207
france 데이터 갯수: 111
georgia 데이터 갯수: 105
greece 데이터 갯수: 32
hungary 데이터 갯수: 10
italy 데이터 갯수: 104
japan 데이터 갯수: 202
luxembourg 데이터 갯수: 11
morocco 데이터 갯수: 20
moldova 데이터 갯수: 18
new zealand 데이터 갯수: 15
peru 데이터 갯수: 25
portugal 데이터 갯수: 26
romania 데이터 갯수: 30
serbia 데이터 갯수: 3
slovakia 데이터 갯수: 6
slovenia 데이터 갯수: 20
turkey 데이터 갯수: 58
uruguay 데이터 갯수: 150
south africa 데이터 갯수: 1166


In [44]:
# 위 함수 결과물과 원하는 연도를 아래 함수에 넣어주세요
for CN in country_32sm[5:]:
    globals()[f'{CN}_df'] = open_csv_year(data_dict[CN], '2022','2021')

길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  1
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이:  0
추가 행 길이: 

In [52]:
for cn in data_dict.keys():
    globals()[f'{cn}_df'].to_csv(f'{cn}.csv')
    

#### country_32
['ARGENTINA',
 'AUSTRALIA',
 'AUSTRIA',
 'BELGIUM',
 'BULGARIA',
 'BRAZIL',
 'CZECH REPUBLIC',
 'SWITZERLAND',
 'CHILE',
 'CROATIA',
 'CYPRUS',
 'GERMANY',
 'SPAIN',
 'FRANCE',
 'GEORGIA',
 'GREECE',
 'HUNGARY',
 'ITALY',
 'JAPAN',
 'LUXEMBOURG',
 'MOROCCO',
 'MOLDOVA',
 'NEW ZEALAND',
 'PERU',
 'PORTUGAL',
 'ROMANIA',
 'SERBIA',
 'SLOVAKIA',
 'SLOVENIA',
 'TURKEY',
 'URUGUAY',
 'SOUTH AFRICA']

In [55]:
argentina_df.columns

Index(['STATION', 'DATE', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'NAME', 'DP01',
       'DP01_ATTRIBUTES', 'DP10', 'DP10_ATTRIBUTES', 'DP1X', 'DP1X_ATTRIBUTES',
       'DYXP', 'DYXP_ATTRIBUTES', 'EMXP', 'EMXP_ATTRIBUTES', 'PRCP',
       'PRCP_ATTRIBUTES', 'DT00', 'DT00_ATTRIBUTES', 'DT32', 'DT32_ATTRIBUTES',
       'DYNT', 'DYNT_ATTRIBUTES', 'EMNT', 'EMNT_ATTRIBUTES', 'TMIN',
       'TMIN_ATTRIBUTES', 'DX32', 'DX32_ATTRIBUTES', 'DX70', 'DX70_ATTRIBUTES',
       'DX90', 'DX90_ATTRIBUTES', 'DYXT', 'DYXT_ATTRIBUTES', 'EMXT',
       'EMXT_ATTRIBUTES', 'TAVG', 'TAVG_ATTRIBUTES', 'TMAX', 'TMAX_ATTRIBUTES',
       'CDSD', 'CDSD_ATTRIBUTES', 'CLDD', 'CLDD_ATTRIBUTES', 'HDSD',
       'HDSD_ATTRIBUTES', 'HTDD', 'HTDD_ATTRIBUTES', 'DSND', 'DSND_ATTRIBUTES',
       'DYSD', 'DYSD_ATTRIBUTES', 'EMSD', 'EMSD_ATTRIBUTES'],
      dtype='object')

In [76]:
# 필요없는 컬럼 삭제
# 유효컬럼 :['STATION', 'DATE', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'NAME','DT00', 'DT00_ATTRIBUTES','DYNT', 'DYNT_ATTRIBUTES', 'EMNT', 'EMNT_ATTRIBUTES', 'TMIN','TMIN_ATTRIBUTES']

argentina_df.drop(columns = ['HTDD', 'HTDD_ATTRIBUTES', 'DSND', 'DSND_ATTRIBUTES',
       'DYSD', 'DYSD_ATTRIBUTES', 'EMSD', 'EMSD_ATTRIBUTES'],inplace= True)

In [92]:
argentina_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1387 entries, 0 to 1386
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   STATION          1387 non-null   object 
 1   DATE             1387 non-null   object 
 2   LATITUDE         1387 non-null   float64
 3   LONGITUDE        1387 non-null   float64
 4   ELEVATION        1387 non-null   float64
 5   NAME             1387 non-null   object 
 6   DT00             1382 non-null   float64
 7   DT00_ATTRIBUTES  1382 non-null   object 
 8   DYNT             1382 non-null   float64
 9   DYNT_ATTRIBUTES  1382 non-null   object 
 10  EMNT             1382 non-null   float64
 11  EMNT_ATTRIBUTES  1382 non-null   object 
 12  TMIN             1382 non-null   float64
 13  TMIN_ATTRIBUTES  1382 non-null   object 
dtypes: float64(7), object(7)
memory usage: 151.8+ KB


In [95]:
# 다시 csv로 대체

argentina_df.to_csv('argentina.csv')

---

In [107]:
for col in austria_df.columns:
    if austria_df[col].isnull().sum() > (len(austria_df) * 0.1) :
        austria_df.drop(columns = [col],inplace = True)

austria_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181 entries, 0 to 180
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   STATION          181 non-null    object 
 1   DATE             181 non-null    object 
 2   LATITUDE         181 non-null    float64
 3   LONGITUDE        181 non-null    float64
 4   ELEVATION        181 non-null    float64
 5   NAME             181 non-null    object 
 6   DX32             176 non-null    float64
 7   DX32_ATTRIBUTES  176 non-null    object 
 8   DX70             176 non-null    float64
 9   DX70_ATTRIBUTES  176 non-null    object 
 10  DX90             176 non-null    float64
 11  DX90_ATTRIBUTES  176 non-null    object 
 12  DYXT             176 non-null    float64
 13  DYXT_ATTRIBUTES  176 non-null    object 
 14  EMXT             176 non-null    float64
 15  EMXT_ATTRIBUTES  176 non-null    object 
 16  TMAX             176 non-null    float64
 17  TMAX_ATTRIBUTES 

---
#### 국가별 필요한 컬럼만 추출

In [112]:
for country in country_32sm[2:]:
    for col in globals()[f'{country}_df'].columns:
        if globals()[f'{country}_df'][col].isnull().sum() > (len(globals()[f'{country}_df']) * 0.1) :
            globals()[f'{country}_df'].drop(columns = [col],inplace = True)

    

C:\Users\roha2\AppData\Local\Temp\ipykernel_2096\1765659762.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  globals()[f'{country}_df'].drop(columns = [col],inplace = True)
C:\Users\roha2\AppData\Local\Temp\ipykernel_2096\1765659762.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  globals()[f'{country}_df'].drop(columns = [col],inplace = True)
C:\Users\roha2\AppData\Local\Temp\ipykernel_2096\1765659762.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

In [116]:
for country in country_32sm:
    print(f'----{country}')
    globals()[f'{country}_df'].to_csv(f'{country}.csv')


----argentina
----australia
----austria
----belgium
----bulgaria
----brazil
----czech republic
----switzerland
----chile
----croatia
----cyprus
----germany
----spain
----france
----georgia
----greece
----hungary
----italy
----japan
----luxembourg
----morocco
----moldova
----new zealand
----peru
----portugal
----romania
----serbia
----slovakia
----slovenia
----turkey
----uruguay
----south africa


---
#### 정합성판별

In [123]:
# 특정 연도의 특정 국가의 데이터 정합성을 편하신 방법으로 확인부탁드립니다.
argentina_df.head()

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,DT00,DT00_ATTRIBUTES,DYNT,DYNT_ATTRIBUTES,EMNT,EMNT_ATTRIBUTES,TMIN,TMIN_ATTRIBUTES
0,AR000000011,2021-01,-30.267,-57.65,53.0,"MONTE CASEROS AERO, AR",0.0,"3,S",20210118.0,",3,S",14.3,"3,S,18,",19.69,"3,,,S"
1,AR000000011,2021-02,-30.267,-57.65,53.0,"MONTE CASEROS AERO, AR",0.0,"5,S",20210217.0,",5,S",14.8,"5,S,17,",18.80,"5,,,S"
2,AR000000011,2021-03,-30.267,-57.65,53.0,"MONTE CASEROS AERO, AR",0.0,"2,S",20210328.0,",2,S",13.8,"2,S,28,",18.17,"2,,,S"
3,AR000000011,2021-04,-30.267,-57.65,53.0,"MONTE CASEROS AERO, AR",0.0,"1,S",20210429.0,",1,S",8.5,"1,S,29,",15.87,"1,,,S"
4,AR000000011,2021-05,-30.267,-57.65,53.0,"MONTE CASEROS AERO, AR",0.0,"2,S",20210513.0,",2,S",4.3,"2,S,13,",8.99,"2,,,S"


In [130]:
argentina_df['DATE'] = pd.to_datetime(argentina_df['DATE'], format='%Y-%m')

In [142]:
import datetime as dt
pv_table = argentina_df.pivot_table(index = 'STATION', columns = argentina_df['DATE'].dt.to_period('M'), values='NAME', aggfunc='count', fill_value=0)

has_all_months = pv_table.apply(lambda row: all(row >= 1), axis=1)
print(has_all_months[has_all_months == True], f'개수 : {len(has_all_months[has_all_months == True])}' )

STATION
AR000087007    True
AR000087129    True
AR000087374    True
AR000875850    True
ARM00087244    True
ARM00087345    True
ARM00087416    True
ARM00087448    True
ARM00087467    True
ARM00087506    True
ARM00087509    True
ARM00087548    True
ARM00087640    True
dtype: bool 개수 : 13


# 이 아래는 디버깅 중
- 위에서 편하게 작업해주세요

In [ ]:
# 특정 국가, 연도가 적용된 결과 df 모아둔 것
# 정확한 코드 아님, 직관적 이해 적은 것
def dfCountryYear(countryList, *years):
    flag = False

    for country in countryList:
        
        print('현재 나라: ', country)
        
        # 특정 국가의 모든 구역 파일명 리스트
        tmpList = open_csv(country)
        
        # 특정 국가의 모든 구역 파일명 리스트와 연도가 적용된 df
        tmp = open_csv_year(tmpList, *years)
    
        # csv 처음 불러왔을 때
        if flag == False:
            print('0')
            # 결과 df 변수에 첫 번째 국가 df를 적용
            df = tmp
            # flag를 변경하여 이 if문에 다시 들어오지 않는다.
            flag = True
            print('df 길이: ', len(df))
        else:
            print('1')
            # 결과 df와 현재 국가 df를 이어붙임
            df = pd.concat([df, tmp], axis = 0)
            print('df 추가 길이: ', len(tmp))
            
    return df

In [ ]:
# 기준 국가
dfResult = dfCountryYear(country_32[1:2], '2022')

len(dfResult)
dfResult

In [ ]:
dfResult

In [ ]:
# 나머지 국가
dfResult = dfCountryYear(country_others)

len(dfResult)
dfResult

In [ ]:
# 추후에 결정되면 저장할때 활용
dfResult.to_csv('data_climate/' + + '.csv', index = False)

# ! 회의 이후에 필요한 내용 추가하여 시각화 해야함
## EDA


### 시각화 참고 자료
- plt.hist
    - https://lifelong-education-dr-kim.tistory.com/entry/%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%9D%98-%EB%B6%84%ED%8F%AC%EB%A5%BC-%EA%B0%80%EC%8B%9C%ED%99%94-%ED%95%98%EB%8A%94-matplotlib-%EB%9D%BC%EC%9D%B4%EB%B8%8C%EB%9F%AC%EB%A6%AC%EC%9D%98-hist

In [ ]:
dfTMP = globals()[koreaList[0]]
dfTMP.info()

In [ ]:
plt.figure(figsize = (10, 8))

dfTMP['TMAX'].hist(color = 'r', alpha = 0.5)
dfTMP['TMIN'].hist(color = 'b', alpha = 0.5)
dfTMP['TAVG'].hist(color = 'g', alpha = 0.5)
plt.show()